# Sesión 13 A

## Naive Bayes

> **Objetivo:** Comprender el modelo de Naive Bayes como un buen punto de partida para benchmark.

### 1. Introducción

El modelo Naive Bayes es un modelo probabilístico generativo utilizado principalmente para clasificación. Su objetivo es modelar la distribución conjunta:

$$P(C, X_1, \ldots, X_n)$$

de manera simple y compacta. Para lograrlo, asume que todas las características son independientes entre sí dado el valor de la clase. Este supuesto, aunque “ingenuo”, reduce drásticamente el número de parámetros y permite inferencia rápida y eficiente.

A pesar de lo fuerte del supuesto de independencia, Naive Bayes funciona sorprendentemente bien en la práctica, especialmente en problemas con muchos atributos y pocos datos. Su estructura probabilística es la siguiente:


![](../images/sesion13-naive.png)

**Figura 1:** Estructura gráfica del modelo Naive Bayes, donde la variable objetivo $C$ influye en cada una de las características $X_i$ de forma independiente.

### 2. Independencia condicional y factorización

El supuesto Naive Bayes establece:

$$ (X_i \perp X_j \mid C) \quad \forall i \neq j $$

Con esta independencia, la distribución conjunta se factoriza como:

$$
P(C, X_1,\ldots,X_n) = P(C)\prod_{i=1}^n P(X_i \mid C)
$$

Esta factorización hace que el modelo sea compacto, fácil de entrenar y escalable incluso con miles de variables.

### 3. Entrenamiento del modelo Naive Bayes

Para entrenar un modelo Naive Bayes a partir de datos, estimamos dos tipos de parámetros:


#### a) Probabilidad previa de la clase

$$
P(C=c)=\frac{\#\text{instancias con clase }c}{\#\text{total de instancias}}
$$

Indica qué tan frecuente es cada clase en el conjunto de datos.


#### b) Probabilidades condicionales de las características

$$
P(X_i=x \mid C=c)=\frac{\#\text{instancias con }X_i=x\text{ y }C=c}{\#\text{instancias con }C=c}
$$

Se calcula **para cada atributo** y **para cada valor posible** del atributo, condicionado a cada clase.

Estas cantidades conforman las tablas CPD del modelo.

### 4. Clasificación con Naive Bayes

Dada una nueva observación:

$$
x = (x_1, \ldots, x_n)
$$

queremos elegir la clase más probable. Aplicamos la regla MAP (Maximum A Posteriori):

$$
\hat{c} = \arg\max_c P(C=c)\prod_{i=1}^n P(x_i \mid C=c)
$$

En este proceso:

- Se usan **todas** las características de la observación.
- Cada una contribuye multiplicando su probabilidad condicional.
- Finalmente elegimos la clase con mayor probabilidad posterior.

### 5. Ejemplo práctico

In [1]:
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
ruta = os.path.join('..', 'data', 'weather.csv')
df = pd.read_csv(ruta)

In [5]:
df.head()

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Rain,Mild,High,False,Yes
4,Rain,Cool,Normal,False,Yes


In [9]:
df.Windy.value_counts()

Windy
False    8
True     6
Name: count, dtype: int64

In [10]:
target = "Play"
features = df.columns.tolist()
features.remove(target)

print("Features:", features)
print("Target:", target)

Features: ['Outlook', 'Temperature', 'Humidity', 'Windy']
Target: Play


En el caso de `Naive Bayes` podemos seguir utilizando la clase `DiscreteBayesianNetwork` de `pgmpy` para definir el modelo, entrenarlo y hacer predicciones.

**Nota las independencias**... en este caso, las variables observadas son independientes entre sí dado el nodo padre (la variable objetivo). 

> En el código esto significa que el nodo padre (la $y$) siempre viene acompañada de todas las demás variables como nodos hijos directos.

In [11]:
weather_model = DiscreteBayesianNetwork([
    ("Play", "Outlook"),
    ("Play", "Humidity"),
    ("Play", "Windy"),
    ("Play", "Temperature")
])

In [12]:
# Train | test
train_df = df.sample(frac=0.7, random_state=42)
test_df = df.drop(train_df.index)
train_df.shape, test_df.shape

((10, 5), (4, 5))

Recordemos que con el método `fit` entrenamos el modelo Naive Bayes.  
Durante este proceso, pgmpy **estima las distribuciones de probabilidad necesarias** para hacer predicciones:

- La probabilidad previa de la clase $$P(C)$$
- Las probabilidades condicionales de cada atributo $$P(X_i \mid C)$$

Todas estas cantidades se calculan mediante **Maximum Likelihood Estimation (MLE)**, es decir, contando frecuencias en los datos.

In [14]:
weather_model.fit(df)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'Outlook': 'C', 'Temperature': 'C', 'Humidity': 'C', 'Windy': 'N', 'Play': 'C'}


In [15]:
print(weather_model.get_cpds(target))

+-----------+----------+
| Play(No)  | 0.357143 |
+-----------+----------+
| Play(Yes) | 0.642857 |
+-----------+----------+


In [16]:
for cpd in weather_model.get_cpds():
    print(cpd)
    print("#-----------------------------#")

+-----------+----------+
| Play(No)  | 0.357143 |
+-----------+----------+
| Play(Yes) | 0.642857 |
+-----------+----------+
#-----------------------------#
+-------------------+----------+--------------------+
| Play              | Play(No) | Play(Yes)          |
+-------------------+----------+--------------------+
| Outlook(Overcast) | 0.0      | 0.4444444444444444 |
+-------------------+----------+--------------------+
| Outlook(Rain)     | 0.4      | 0.3333333333333333 |
+-------------------+----------+--------------------+
| Outlook(Sunny)    | 0.6      | 0.2222222222222222 |
+-------------------+----------+--------------------+
#-----------------------------#
+------------------+----------+--------------------+
| Play             | Play(No) | Play(Yes)          |
+------------------+----------+--------------------+
| Humidity(High)   | 0.8      | 0.3333333333333333 |
+------------------+----------+--------------------+
| Humidity(Normal) | 0.2      | 0.6666666666666666 |
+------

Ya que tenemos las **CPDs**, entonces podemos proceder a hacer `predict` sobre el conjunto de prueba `test_df`.

In [17]:
predictions = weather_model.predict(test_df.drop(columns=[target]))

  0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
predictions

,Outlook,Temperature,Humidity,Windy,Play
3,Rain,Mild,High,False,Yes
6,Overcast,Cool,Normal,True,Yes
7,Sunny,Mild,High,False,Yes
10,Sunny,Mild,Normal,True,Yes


In [19]:
# Mask de resultados correctos
predictions['Play'].values == test_df['Play'].values

array([ True,  True, False,  True])

In [20]:
# Accuracy
(predictions['Play'].values == test_df['Play'].values).mean()

np.float64(0.75)

#### 5.1 Ejemplo con clase ``NaiveBayes``

Aquí dejo otro ejemplo de cómo hacerlo usando la clase `NaiveBayes` y luego construyendo un `DiscreteBayesianNetwork` equivalente. 

In [21]:
from sklearn.model_selection import train_test_split
from pgmpy.models import NaiveBayes, DiscreteBayesianNetwork
from pgmpy.inference import VariableElimination

In [22]:
target = "Play"
train, test = train_test_split(df, test_size=0.3, random_state=42)

Aquí primero instanciamos el modelo `NaiveBayes` y lo entrenamos con el conjunto de entrenamiento

In [24]:
# entrenar modelo NB
naive_bayes = NaiveBayes()
naive_bayes.fit(train, parent_node=target)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'Outlook': 'C', 'Temperature': 'C', 'Humidity': 'C', 'Windy': 'N', 'Play': 'C'}


In [25]:
# construir modelo DiscreteBayesianNetwork 
bayesian_network = DiscreteBayesianNetwork(
    [(target, f) for f in df.columns if f != target]
)

In [26]:
# añadir CPDs aprendidas desde model.cpd
for cpd in naive_bayes.cpds:
    bayesian_network.add_cpds(cpd)

bayesian_network.check_model()

True

In [27]:
# Instanciamos el motor de inferencia
infer = VariableElimination(bayesian_network)

In [28]:
# Predicción (aquí usamos MAP "a mano" | equivalente a predict)
y_true = []
y_pred = []

for _, row in test.iterrows():
    evidence = row.drop(target).to_dict()
    q = infer.map_query(variables=[target], evidence=evidence, show_progress=False)
    y_true.append(row[target])
    y_pred.append(q[target])

In [29]:
# accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_true, y_pred))

Accuracy: 0.8


### 6. Conclusiones

- Naive Bayes es simple, pero muy efectivo.  
- Su poder proviene del supuesto de independencia condicional.  
- Entrenar el modelo equivale a contar frecuencias.  
- Clasificar consiste en evaluar la probabilidad posterior para cada clase y elegir la mayor.  
- Es escalable, rápido y funciona muy bien en problemas con muchos atributos y pocos datos.